<a href="https://colab.research.google.com/github/alektebel/alektebel.github.io/blob/main/Diego_Rodr%C3%ADguez_AG1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad guiada 1

In [ ]:
url_gh_AG1 = https://github.com/alektebel/03MAIR--Algoritmos-de-optimizacion--2024/tree/main/AG1